Inspired by:

https://github.com/google-deepmind/sonnet/blob/v2/sonnet/src/nets/vqvae.py

Config values from paper plus:

https://github.com/google-deepmind/sonnet/blob/v2/examples/vqvae_example.ipynb

In [ ]:
!pip install -q wandb pytorch_lightning

In [ ]:
import argparse
import math
import torch
import torch.utils.data
import torchvision
import wandb
import pytorch_lightning as pl

from __future__ import print_function
from collections import OrderedDict
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor, EarlyStopping
from pytorch_lightning.tuner import Tuner
from torch.optim.lr_scheduler import CosineAnnealingLR, OneCycleLR


pl.seed_everything(42)

# Data

In [ ]:
class CIFAR10DataModule(pl.LightningDataModule):
    def __init__(self, config):
        super().__init__()
        self.batch_size = config.batch_size
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

    def prepare_data(self):
        torchvision.datasets.CIFAR10(root='./data', train=True, download=True)
        torchvision.datasets.CIFAR10(root='./data', train=False, download=True)

    def setup(self, stage=None):
        if stage in ('fit', None):
            self.cifar10_train = torchvision.datasets.CIFAR10(
                root='./data', train=True, transform=self.transform)
            self.cifar10_val = torchvision.datasets.CIFAR10(
                root='./data', train=False, transform=self.transform)

    def train_dataloader(self):
        return torch.utils.data.DataLoader(
            self.cifar10_train,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=2, # TODO: check if we need more workers
            pin_memory=True
        )

    def val_dataloader(self):
        return torch.utils.data.DataLoader(
            self.cifar10_val,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=2, # TODO: check if we need more workers
            pin_memory=True
        )

# Model

In [ ]:
class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        '''
        "[...] followed by two residual 3 × 3 blocks (implemented as ReLU, 3x3 conv,
        ReLU, 1x1 conv), all having 256 hidden units."
        '''
        super().__init__()
        if in_channels != out_channels:
            self.identity = nn.Conv2d(
                in_channels=in_channels,
                out_channels=out_channels,
                kernel_size=1
            )
        else:
            self.identity = nn.Identity()

        self.block = nn.Sequential(
            nn.Conv2d(
                in_channels=in_channels,
                out_channels=in_channels,
                kernel_size=3,
                padding=1
            ),
            nn.ReLU(),
            nn.Conv2d(
                in_channels=in_channels,
                out_channels=out_channels,
                kernel_size=1
            )
        )
        self.res_act = nn.ReLU()

    def forward(self, x):
        out = self.block(x) + self.identity(x)
        return self.res_act(out)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, config):
        '''
        "The encoder consists of 2 strided convolutional layers with stride 2 and
        window size 4 × 4, followed by two residual 3 × 3 blocks (implemented as ReLU,
        3x3 conv, ReLU, 1x1 conv), all having 256 hidden units."
        '''
        super().__init__()
        self.strided_blocks = nn.Sequential(*[
            nn.Sequential(
                nn.Conv2d(
                    in_channels=config.in_channels if i == 0 else config.hidden_channels,
                    out_channels=config.hidden_channels,
                    kernel_size=4,
                    stride=2,
                    padding=1
                ),
                nn.ReLU()
            ) for i in range(config.nlayers)
        ])

        self.res_blocks = nn.Sequential(*[
            ResBlock(
                in_channels=config.hidden_channels,
                out_channels=config.hidden_channels if i < config.nblocks-1 else config.latent_channels
            ) for i in range(config.nblocks)
        ])

    def forward(self, x):
        x = self.strided_blocks(x)
        x = self.res_blocks(x)
        return x

In [ ]:
class Quantizer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.codebook = nn.Embedding(config.codebook_size, config.latent_dim)

    def forward(self, inputs):
        inputs = inputs.permute(0, 2, 3, 1).contiguous()
        flat_inputs = inputs.reshape(-1, self.config.latent_dim)

        # Σ(x-y)^2 = Σx^2 - 2xy + Σy^2
        dists = (
            torch.sum(flat_inputs ** 2, dim=1, keepdim=True) - # Σx^2
            2 * (flat_inputs @ self.codebook.weight.t()) +     # 2*xy
            torch.sum(self.codebook.weight ** 2, dim=1)        # Σy^2
        )

        enc_idxs = torch.argmin(dists, dim=1)
        quantized_inputs = self.codebook(enc_idxs).reshape(inputs.shape)

        # "The VQ objective uses the l2 error to move the embedding vectors
        # e_i towards the encoder outputs z_e(x)"
        embedding_loss = F.mse_loss(quantized_inputs, inputs.detach())
        
        # "since the volume of the embedding space is dimensionless, it can grow
        # arbitrarily if the embeddings e_i do not train as fast as the encoder
        # parameters. To make sure the encoder commits to an embedding and its
        # output does not grow, we add a commitment loss"
        commitment_loss = F.mse_loss(quantized_inputs.detach(), inputs)
        
        # parts 2 & 3 of full loss (ie. not including reconstruciton loss)
        vq_loss = commitment_loss * self.config.embed_loss_beta + embedding_loss

        # sets the output to be the input plus the residual value between the
        # quantized latents and the inputs like a resnet for Straight Through
        # Estimation (STE)
        quantized_inputs = inputs + (quantized_inputs - inputs).detach()
        quantized_inputs = quantized_inputs.permute(0, 3, 1, 2).contiguous()

        return (quantized_inputs, vq_loss, embedding_loss, commitment_loss)

In [ ]:
class Decoder(nn.Module):
    def __init__(self, config):
        '''
        "The decoder similarly has two residual 3 × 3 blocks, followed by
        two transposed convolutions with stride 2 and window size 4 × 4"
        '''
        super().__init__()

        self.res_blocks = nn.Sequential(*[
            ResBlock(
                in_channels=config.latent_channels if i==0 else config.hidden_channels,
                out_channels=config.hidden_channels
            ) for i in range(config.nblocks)
        ])

        self.transposed_blocks = nn.Sequential(*[
            nn.Sequential(
                nn.ConvTranspose2d(
                    in_channels=config.hidden_channels,
                    out_channels=config.hidden_channels,
                    kernel_size=4,
                    stride=2,
                    padding=1
                ),
                nn.ReLU()
            ) for _ in range(config.nlayers-1)
        ])

        self.out_layer = nn.ConvTranspose2d(
            in_channels=config.hidden_channels,
            out_channels=config.in_channels,
            kernel_size=4,
            stride=2,
            padding=1
        )

    def forward(self, z_q):
        out = self.res_blocks(z_q)
        out = self.transposed_blocks(out)
        out = self.out_layer(out)
        return out

In [ ]:
class VQVAE(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.encoder =   Encoder(config)
        self.decoder =   Decoder(config)
        self.quantizer = Quantizer(config)
        self.config =    config

    def encode(self, x):
        return self.encoder(x)

    def quantize(self, z):
        return self.quantizer(z)

    def decode(self, q_z):
        return self.decoder(q_z)

    def loss(self, x_hat, x, quantized):
        MSE = F.mse_loss(x_hat, x)
        loss = MSE + quantized['vq_loss']
        
        return {
            'MSE':  MSE,
            'loss': loss,
            **quantized
        }

    def forward(self, x):
        z = self.encode(x)
        quantized = self.quantize(z)
        x_hat = self.decode(quantized['quantized_inputs'])
        losses = self.loss(x_hat, x, quantized)
        
        return {'x_hat': x_hat, **losses}

# Lightning Model

In [ ]:
class LitVQVAE(pl.LightningModule):
    def __init__(self, model, config):
        super().__init__()
        self.model = model
        self.config = config
        self.lr = config.lr

        if self.logger:
            self.logger.experiment.config.update(config)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, _ = batch
        out = self(x)

        self.log('train/loss',            out['loss'],            prog_bar=True)
        self.log('train/MSE',             out['MSE'],             prog_bar=True)
        self.log('train/vq_loss',         out['vq_loss'],         prog_bar=True)
        self.log('train/embedding_loss',  out['embedding_loss'],  prog_bar=True)
        self.log('train/commitment_loss', out['commitment_loss'], prog_bar=True)

        return out['loss']

    def validation_step(self, batch, batch_idx):
        x, _ = batch
        out = self(x)

        self.log('val/loss',            out['loss'],            prog_bar=True)
        self.log('val/MSE',             out['MSE'],             prog_bar=True)
        self.log('val/vq_loss',         out['vq_loss'],         prog_bar=True)
        self.log('val/embedding_loss',  out['embedding_loss'],  prog_bar=True)
        self.log('val/commitment_loss', out['commitment_loss'], prog_bar=True)

        if batch_idx == 0:
            n_images = min(x.size(0), 8)
            comparison = torch.cat([x[:n_images], out['x_hat'][:n_images]])
            grid = torchvision.utils.make_grid(comparison)
            self.logger.experiment.log({"val/reconstructions": [wandb.Image(grid, caption="Top: Original, Bottom: Reconstructed")]})

        return out['loss']

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(
            self.parameters(),
            lr=self.lr,
            betas=(self.config.beta1, self.config.beta2),
            weight_decay=self.config.weight_decay
        )
        
        if self.config.use_lr_schedule:
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=self.config.max_epochs)
            return [optimizer], [scheduler]
        
        return optimizer


# Config

In [ ]:
def get_num_downsample_layers(img_size):
    """
    Get the number of strided Conv2D layers
    required to produce a 2x2 output volume
    """
    if img_size < 2:
        raise ValueError("Image size must be at least 2x2.")

    # Calculate the minimum number of downsample layers required for 2x2 final
    num_layers = math.ceil(math.log2(img_size / 2))
    return num_layers

def build_channel_dims(start_channels, nlayers):
    """
    Construct a list of channel counts for nlayers downsample layers
    assuming the channels double as spatial dims halve
    """
    channels = []
    for _ in range(nlayers):
        channels.append(start_channels)
        start_channels *= 2
    return channels

class CIFAR10VQVAEConfig:
    def __init__(self):
        # model checkpoints
        self.checkpoint_path = "./checkpoints"
        self.save_top_k = 1
        # training
        self.batch_size = 128
        self.max_epochs = 120
        self.training_steps = 250000
        self.num_workers = 2
        # optimizer
        self.lr = 2e-4
        self.beta1 = 0.9
        self.beta2 = 0.999
        self.weight_decay = 0.0 # 1e-2
        self.use_wd_schedule = False
        self.use_lr_schedule = False
        # input properties
        self.img_size = 32
        self.in_channels = 3
        # latents / quantization
        self.latent_channels = 64
        self.codebook_size = 512
        self.embed_loss_beta = 0.25
        # encoder/decoder
        self.hidden_channels = 128
        self.nblocks = 2
        self.nlayers = 2

    def update(self, updates):
        for key, value in updates.items():
            if hasattr(self, key):
                setattr(self, key, value)
        self.nlayers = get_num_downsample_layers(self.img_size)
        self.channel_dims = build_channel_dims(self.start_channels, self.nlayers)

    def to_dict(self):
        return {k: v for k, v in self.__dict__.items()}

# COMPONENT TESTING

In [ ]:
dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, num_workers=1)

In [ ]:
img, label = next(iter(dataloader))
img.shape

In [ ]:
config = CIFAR10VQVAEConfig()

In [ ]:
encoder = Encoder(config)

In [ ]:
z = encoder(img)
print(z.shape)

In [ ]:
quantizer = Quantizer(config)

In [ ]:
z_q, vq_loss, embedding_loss, commitment_loss = quantizer(z)
print(z_q.shape)

In [ ]:
decoder = Decoder(config)

In [ ]:
x_hat = decoder(z_q)
print(x_hat.shape)

In [ ]:
vqvae = VQVAE(config)

In [ ]:
vqvae(img)

# Train

In [ ]:
config = CIFAR10VQVAEConfig()
model = VQVAE(config)
lit_model = LitVQVAE(model, config)
cifar10_data = CIFAR10DataModule(config)

wandb.init(project="VQ-VAE CIFAR-10", config=config.to_dict())
wandb_logger = WandbLogger(project="VQ-VAE CIFAR-10", log_model=False)
wandb_logger.watch(lit_model, log="all")

lr_monitor = LearningRateMonitor(logging_interval='step')

checkpoint_callback = ModelCheckpoint(
    dirpath=config.checkpoint_path,
    filename='model-{epoch:02d}-{val_loss:.2f}',
    every_n_epochs=5,
    save_top_k=config.save_top_k,
    monitor='val/loss',
    mode='min',
    save_last=True
)

# Define the EarlyStopping callback
early_stop_callback = EarlyStopping(
    monitor='val/loss',
    min_delta=0.00,
    patience=3,
    verbose=True,
    check_finite=True
)

In [ ]:
#wandb.finish()

In [ ]:
trainer = pl.Trainer(
    max_epochs=config.max_epochs,
    devices=1,
    accelerator="gpu",
    precision="16-mixed",
    logger=wandb_logger,
    callbacks=[
        lr_monitor,
        early_stop_callback,
        # checkpoint_callback
    ],
    log_every_n_steps=1,
    # overfit_batches=1,
)

# tuner = Tuner(trainer)
# tuner.lr_find(lit_model, datamodule=cifar10_data)

In [ ]:
trainer.fit(lit_model, cifar10_data)
wandb.finish()

# Sweeps

In [ ]:
sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'val/loss',
        'goal': 'minimize'
    },
    'parameters': {
        'nblocks': {
            'values': [0, 1, 2]
        },
        'latent_dim': {
            'values': [128, 256, 512]
        },
        'start_channels': {
            'values': [32, 64, 128]
        },
        'lr': {
            'min': 1e-5,
            'max': 1e-2,
            'distribution': 'uniform'
        },
        'beta1': {
            'values': [0.9, 0.95, 0.99]
        },
        'beta2': {
            'values': [0.999, 0.9999]
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project="VQVAE CIFAR-10")

In [ ]:
def train():
    with wandb.init() as run:
        config = CIFAR10VQVAEConfig()
        config.update(wandb.config)

        model = VQVAE(config)
        lit_model = LitVQVAE(model, config)
        cifar10_data = CIFAR10DataModule(config)

        wandb_logger = WandbLogger(project="VQVAE CIFAR-10", log_model=False)

        early_stop_callback = EarlyStopping(
            monitor='val/loss',
            min_delta=0.00,
            patience=3,
            verbose=True,
            check_finite=True
        )

        trainer = pl.Trainer(
            max_epochs=config.max_epochs,
            devices=1,
            accelerator="gpu",
            precision="16-mixed",
            logger=wandb_logger,
            callbacks=[early_stop_callback]
        )

        trainer.fit(lit_model, cifar10_data)

In [ ]:
wandb.agent(sweep_id, train, count=5)